In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# .env 파일에 LANGCHAIN_API_KEY를 입력합니다.
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("Big_Pro_02")

LangSmith 추적을 시작합니다.
[프로젝트명]
Big_Pro_02


In [2]:
import os 
from langchain_teddynote.messages import stream_response

from utils.RAGPipeLine import Ragpipeline
from utils.mongo_RAGdoc import RagDocument

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
pipeline = Ragpipeline()

[초기화] vector_store 초기화 완료
[초기화] retriever 초기화 완료
[초기화] RAG chain 초기화 완료


/home/khw/anaconda3/envs/bigproject/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


In [4]:
obj = RagDocument()
# obj.file.path = './documents/크롤링_리스트.csv'
obj.file.path = './documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf'

In [5]:
with open(obj.file.path, 'rb') as f:
    if pipeline.update_vector_db(f, obj.file):
        print("[문서업로드] 벡터 스토어 업데이트에 성공하였습니다.")
    else:
        # 유사한 문서가 존재하여 업데이트에 실패 시 저장된 문서 삭제
        print("[문서업로드] 유사한 문서가 발견되어 벡터스토어 업데이트가 거절되었습니다.")


유사도 검사 중...:  25%|██▌       | 1/4 [00:00<00:01,  1.81it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.0


유사도 검사 중...:  50%|█████     | 2/4 [00:00<00:00,  2.44it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.0


유사도 검사 중...:  75%|███████▌  | 3/4 [00:01<00:00,  2.62it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.0


유사도 검사 중...: 100%|██████████| 4/4 [00:01<00:00,  2.39it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 3.884106283755847e-06
모두 유사한 청크로 판단되어 해당 문서가 저장되지 않음
[문서업로드] 유사한 문서가 발견되어 벡터스토어 업데이트가 거절되었습니다.


### 1.1 확인해보기 - DB 접근 및 질문에 대한 유사도 확인 

In [6]:
vectorDB = pipeline.init_vectorDB()

# 특정 데이터베이스에 접근하고 싶을때
# persist_dir = "./database"        
# vectorDB = pipeline.init_vectorDB(persist_dir)        

In [7]:
retriever = pipeline.init_retriever()

In [ ]:
query = '인도 통관 및 운송'

docs = retriever.get_relevant_documents(query)

In [37]:
# 문서 조회1
query = '인도 통관 및 운송'   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = vectorDB.similarity_search(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴

print('-'*50)
for doc in result:
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print('---'*50)
    print(f"출처 : {doc.metadata['source'].split('/')[-1]} - page : {doc.metadata['page']}")

--------------------------------------------------
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물
품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도
관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되
며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를
완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage
Charge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. 
 
2) 임시 통관 
 
인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해
당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반
출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. 
 
  ㅇ ATA Carnet 
    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩
을 뜻하는 불어 Carnet의 머리글자 조합으로, 임시 통관을 위한 수첩을 말한다. ATA 협약에는 인도를 포함하여 전 세계 71개국이 서명하

In [38]:
# 문서 조회1
query = '인도 통관 및 운송'   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = vectorDB.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('---'*50)
for doc in result:
    print(f"문서 내용: {doc[0].page_content}") # 문서 내용 표시
    print('---'*50)
    print(f"출처 : {doc[0].metadata['source'].split('/')[-1]} - page : {doc[0].metadata['page']}")
    print(f"유사도 : {doc[1]}")

[(Document(page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해\n당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반\n출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. \n \n  ㅇ ATA Carnet \n    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩\n을 뜻하는 불어 Carnet의 머리글자 조합으로, 임시 통관을 위한 수첩을 말한다. ATA 협약에는 인도를 포함하여 전 세계 71개국이 서명하\n여 발효 중

In [39]:
question = "인도 통관 및 운송에 대해서 알려줘."

results = vectorDB.similarity_search_with_score(question, k=1)

results[0][1]

0.27029422059062225

In [19]:
question = "럭키비키가 무엇인지 알려줘."

results = vectorDB.similarity_search_with_score(question, k=1)

results[0][1]

0.4358286053657968

In [20]:
question   = '인도 통관 및 운송에 대해서 알려줘.'
session_id = 'abc123'
answer = pipeline.chat_generation(question, session_id)

print(answer)

Parent run b9853a55-50b5-40bf-b8ae-4313eb49766b not found for run eec58dec-e9d7-4406-b684-f02c2133826b. Treating as a root run.


0.27029422059062225
[히스토리 생성] 새로운 히스토리를 생성합니다. 세션 ID: abc123


Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


{'input': '인도 통관 및 운송에 대해서 알려줘.', 'chat_history': [], 'context': [Document(page_content='5. 통관 및 운송\n \n \n가. 통관제도\n  \n \n통관 유형별 절차\n \n1) 정식통관 \n \n인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 항공 운송은 약 1~2 근무일이 소요된다. 일단 물\n품이 입항하게 되면 보세구역장치에 적하되고, 이후 전자데이터 교환(EDI) 신고 또는 수작업신고 통해 수입신고를 하게 된다. 이후 인도\n관세청의 수입요건, 관세 평가 등을 검사하는 Appraiser Section과 관련 서류 및 관세율을 재점검하는 Audit Section 과정을 거치게 되\n며, 이를 통과하면 모든 서류에 서명 받게 된다. 이후 Challan No.라는 인도 관세 납부를 위한 번호를 부여받게 되고, 이를 근거로 관세를\n완납하게 되면 물품을 반출할 수 있다. 또한, 정식통관의 경우 인도 세관은 일반적으로 화물 도착 후 7일 이후부터 체화료(Demurrage\nCharge)를 부과하고, 컨테이너 운송의 경우 5일 이후부터 항공운송인 경우 3일 이후부터 체화료가 부과한다. \n \n2) 임시 통관 \n \n인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해서는 해\n당 품목의 관세를 납부해야하는데, 다시 반출할 시 장치했던 시간에 비례하여 관세를 환급받을 수 있다. 인도 내로 반입되지 않고 바로 반\n출할 경우 최대 98%까지 환급이 가능하나, 인도 내 해당 물품이 사용될 경우 기간과 상관없이 관세의 환급이 불가하다. \n \n  ㅇ ATA Carnet \n    - ATA 까르네(ATA Carnet)는 Admission Temporaire이라는 임시통관을 뜻하는 불어와 Temporary Admission의 영어 그리고 수첩\n을 뜻하는 불어 Carnet의 머리글자 조

In [25]:
print(answer['answer'])
# stream_response(answer['answer'])

인도의 통관 및 운송에 대한 정보는 다양한 유형의 통관 절차와 주요 포워딩 업체 및 통관업체에 대한 정보를 포함하고 있습니다. 

1. 통관 절차:
- 정식통관: 일반적인 경우 3~4 근무일이 소요되며, 관세평가, 관세율 재점검 등의 과정을 거칩니다.
- 임시통관: 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 이루어집니다.
- 전시물품의 통관: 전시·박람회, 학회 등을 위한 물품의 통관은 관세가 면제되지만 일정 조건을 충족해야 합니다.
- 약식통관: 2,000루피 이하의 제품의 경우 약식통관이 가능하며, 세관에 제출해야 할 서류가 있습니다.
- 허가 통관: 수입의 제한 품목이나 식물, 씨앗 등의 경우 허가 통관이 필요합니다.

2. 유의사항:
- 통관의 지연: 인도의 통관은 행정상 3~4 근무일이 소요되며, 행정 시스템이 완전하지 않아 통관 지연이 발생할 수 있습니다.
- 통관 정보의 불투명: 관세 행정이 불투명하며, 실무선의 재량권이 크고 융통성 없는 처리 관행으로 인해 애로가 발생할 수 있습니다.
- FTA 사후 협정 적용 불가능: FTA 사후 협정 적용이 어려울 수 있으며, 관세 혜택이 주어지는 경우에도 증빙이 충분치 않아 통관이 거부될 수 있습니다.
- 인도 지역별 세관관할: 지역별 차이나 관할 세관 당국 간 소통 미비로 인하여 애로 해소가 발생할 수 있습니다.
- 서류의 수정: 서류에 오류가 있을 시 수정이 어려울 수 있으며, 허위신고로 인해 과태료가 발생할 수 있습니다.

3. 주요 포워딩 업체 및 통관업체:
- KGL Network Private Limited
- Taewoong Logistics
- CJ Korea Express India Pvt. Ltd
- FIRST FLIGHT
- Gati Ltd.

이 정보를 참고하여 인도에서의 통관 및 운송에 대한 이해를 높이시기 바랍니다.

In [23]:

print('근거:')
# 근거 추출
for data in answer['context']:
    print(data.metadata['source'], ' - page: ', data.metadata['page'])


근거:
./documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf  - page:  0
./documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf  - page:  3
./documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf  - page:  1
./documents/[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf  - page:  2


In [13]:
question   = '인도 통관 및 운송'
titles = pipeline.title_generation(question, "3")           # 제목 개수 정할 수 있음

print(titles)

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


- 인도 통관 절차 및 유의사항
- 인도 통관 시스템의 문제점과 개선 방안
- 인도 통관 업체 소개 및 비교분석


In [14]:
post = pipeline.post_generation(titles.split('-')[1])
    
print(post)

Number of requested results 5 is greater than number of elements in index 4, updating n_results = 4


인도 통관 및 운송: 최신 정보와 유의사항

인도의 통관 절차는 다양한 유형으로 나뉘어져 있으며, 각각의 특징과 유의사항이 존재한다. 정식통관, 임시통관, 전시물품의 통관, 약식통관, 허가 통관 등이 있다. 정식통관의 경우, 통관에는 일정한 시간이 소요되며, 관세청의 수입요건 및 관세평가 등을 거쳐 관세를 납부해야 한다. 임시통관은 품목을 사용하지 않고 반출할 목적이 있는 경우에 이루어지며, 관세를 납부하고 환급을 받을 수 있다. 전시물품의 통관은 관세가 면제되지만 일정한 조건을 충족해야 한다. 약식통관은 2,000루피 이하의 제품에 적용되며, 제품의 종류에 따라 HS코드가 적용된다.

통관 시 유의사항에는 몇 가지 주의해야 할 점이 있다. 인도의 통관은 일반적으로 3~4 근무일이 소요되며, 행정 시스템이 완전하지 않아 통관 지연이 발생할 수 있다. 또한, 관세 정보의 불투명성과 FTA 사후 협정 적용의 어려움, 지역별 세관관할의 차이 등이 통관 애로의 원인이 될 수 있다. 서류의 수정이 어려워 통관 절차가 지연되는 경우도 발생할 수 있다.

주요 포워딩 업체 및 통관업체로는 KGL Network Private Limited, Taewoong Logistics, CJ Korea Express India Pvt. Ltd, FIRST FLIGHT, Gati Ltd. 등이 있다. 이들 업체는 인도 내에서 다양한 물류 및 통관 서비스를 제공하고 있으며, 한국과의 협력을 통해 원활한 무역 활동을 지원하고 있다.

이 게시물은 제약산업정보포털에서 제공된 '인도 통관 및 운송' 문서를 참고하여 작성되었습니다. (출처: 제약산업정보포털, 2019.04.08, 인도 통관 및 운송.pdf)
